## **Laboratorium 13 - Metoda podziału i ograniczeń – algorytm Little’a dla zagadnienia komiwojażera**
  
### Barbara Pobiedzińska, 400781
### Agata Semerjak, 402426  
### Monika Sukiennik, 401060  
### Maciej Wieloch, 303080  
gr 1a, środa 14:45  
02.06.2021

**Zadanie 1.**  
  
Należało zaimplementować algorytm Little'a dla problemu TSP o rozmiarze n >= 6. Naszą daną wejściową była macierz kosztów reprezentowana za pomocą elementu array z biblioteki numpy.  
Poniżej zamieszczamy naszą implementacje algorytmu wraz z opisem.

In [1]:
# funkcja służąca do printowania macierzy
def print_matrix(M):
    print("%3s" % ' ', end='')
    print("%3s" % ' | ', end='')
    for i in range(len(M)):
        print("%7s" % str(i), end='')
    print("\n")
    print("-"*(7*len(M) + 6))
    for r in range(len(M)):
        print("%3s" % str(r), end='')
        print("%3s" % ' | ', end='')
        for c in range(len(M)):
            print("%7s" % str(M[r, c]), end='')
        print("\n")

1. Funkcja odpowiedzialna za redukcję macierzy:

In [2]:
import numpy as np
import sys
inf = np.inf

def reduction(mat):
    global LB
    reduced_mat = mat.copy() # wykonujemy kopię macierzy wejściowej, by wprowadzone zmiany nie zmodyfikowały oryginału
    min_rows = np.amin(reduced_mat, axis=1) # obliczamy minimalne wartości dla każdego wiersza
    for i in range(len(min_rows)):
        reduced_mat[i] -= min_rows[i] # od każdego wiersza odejmujemy minamalną wartość w nim znalezioną
    reduced_mat = reduced_mat.transpose() # transponujemy macierz, by wykonać tę samą operację na kolumnach

    min_cols = np.amin(reduced_mat, axis=1) # obliczamy minimalne wartości dla każdej kolumny
    for i in range(len(min_cols)):
        reduced_mat[i] -= min_cols[i] # od każdej kolumny odejmujemy minamalną wartość w niej znalezioną
    reduced_mat = reduced_mat.transpose() # dokonujemy ponownej transpozycji macierzy, by przywrócić ją do stanu początkowego

    lower_bound = min_rows.sum() + min_cols.sum() # obliczmy dolne ograniczenie dla tego kroku - suma najmniejszych wartości w rzędach i kolumnach (w wejściowej macierzy)
    LB = LB + lower_bound # modyfikujemy zmienną globalną przechowująca dolne ograniczenie

    return reduced_mat

2. Funkcja odpowiedzialna za wybór przejścia:

In [3]:
def find_max(reduced_mat):
    id = -1
    max_table = [] # tablica na maksymalne wartości 
    for i in range(reduced_mat.shape[0]): # iteracja po wierszach macierzy
        for j in range(reduced_mat.shape[1]): # iteracja po kolumnach macierzy
            if reduced_mat[i, j] == 0: # rozpatrujemy jedynie elementy równe zero
                id += 1
                max_table.append([(i,j), 0]) # zapamiętujemy współrzędne znalezionego zera

                mins_row = reduced_mat[i]
                mins_row = np.delete(mins_row, j)
                min_in_row = min(mins_row) # szukamy minimalnej wartości w rzędzie, w którym znajduje się nasze zero
                reduced_mat = reduced_mat.transpose() # trasponujemy, by zrobić to samo dla kolumny

                mins_col = reduced_mat[j]
                mins_col = np.delete(mins_col, i)
                min_in_col = min(mins_col) # szukamy minimalnej wartości w kolumnie, w której znajduje się nasze zero

                reduced_mat = reduced_mat.transpose() # transponujemy ponownie, by powrócić do prawidłowej postaci
                max_table[id][1] = (min_in_col + min_in_row) # jako koszt sumujemy znalezione minimalne wartości znalezione w rozpatrywanym wierszu i kolumnie

    max_val = max_table[0][1]
    id_row = max_table[0][0][0]
    id_col = max_table[0][0][1]
    for i in range(1, len(max_table)):
        if max_table[i][1] > max_val: # wśród wszystkich kosztów szukamy teraz największego
            max_val = max_table[i][1]
            id_row = max_table[i][0][0]
            id_col = max_table[i][0][1]
    return int(id_row), int(id_col), max_val # zwracamy znaleziony największy koszt oraz odpowiadające mu wierz i kolumnę

3. Funkcja aktualizująca macierz kosztów:

In [4]:
def update_matrix(reduced_mat, find_max_result):
    id_row, id_col, max_val = find_max_result # rozpakowanie krotki zwróconej przez poprzednią funkcję

    reduced_mat[id_col][id_row] = inf # zabronienie przejścia w powrotną stronę

    reduced_mat = np.delete(reduced_mat, id_col, 1) # usunięcie z macierzy dnaego wiersza
    reduced_mat = np.delete(reduced_mat, id_row, 0) # usunięcie z macierzy danej kolumny

    reduced_mat = reduction(reduced_mat) # ponownie wykonanie redukcji zaktualizowanej macierzy

    return reduced_mat # zwracamy zaktualizowaną zredukowaną macierz

4. Funkcja sortująca zwróconą przez główny algorytm ścieżkę:

In [5]:
def sort_path(path):
    final_path = [] # tablica przechowująca posortowaną ścieżkę
    unique1 = 0 # w celu wyszukania elementów z początku i końca ścieżki
    unique2 = 0
    for i in range(len(path)): # iteracja po pierwotnie nieuporządkowanej ścieżce
        elem1 = path[i][0]
        elem2 = path[i][1]
        is_unique_1 = True
        is_unique_2 = True

        for j in range(len(path)): # poszukiwanie elementu, który się nie powtarza (tj jest tylko początkiem lub tylko końcem)
            if i == j: # pominięcie dokładnie tego samego elementu
                continue
            if elem1 == path[j][1]: # znaleziono powtórzenie
                is_unique_1 = False
            if elem2 == path[j][0]:
                is_unique_2 = False
        if is_unique_1: # jezeli wyszukany element sie nie powtarza
            unique1 = elem1
        if is_unique_2:
            unique2 = elem2

    path.append((unique2, unique1)) #dopisanie do ścieżki krawędzi łączącej pierwszy i ostatni element
    # sortowanie
    piece = path.pop(0)
    final_path.append(piece)

    while path: 
        last_elem = final_path[-1][1]
        piece = path[0]
        i = 0
        while piece[0] != last_elem: #wyszukanie elementu który zaczyna się tak jak na razie ścieżka się kończy
            i += 1
            piece = path[i]

        piece = path.pop(i)
        final_path.append(piece)

    return final_path

5. Główna funkcja obsługująca cały algorytm:

In [6]:
def TSP(matrix):
    global LB
    LB = 0 # początkowa wartość dolnego ograniczenia

    result = matrix.copy()

    result = reduction(result) # redukcja macierzy
    result_max = find_max(result) # znalezienie odpowiedniej krawędzi

    rows_tab = [i for i in range(len(result))] # tablice przechowujące pierwotną prawidłową indeksację  wierzchołków
    cols_tab = [i for i in range(len(result))]

    path = [(rows_tab.pop(result_max[0]), cols_tab.pop(result_max[1]))]

    while len(result) != 2: # dopóki macierz nie zostanie sprowadzona do rozmiaru 2x2
        print("\nMacierz\n", result)
        print("LB:", LB)
        result = update_matrix(result, result_max) # aktualizacja macierzy kosztów
        result_max = find_max(result) # znalezienie odpowiedniego przejścia
        path.append((rows_tab.pop(result_max[0]), cols_tab.pop(result_max[1]))) # uzupełnienie ścieżki

    print("\nMacierz\n", result)
    print("LB:", LB)

    path = sort_path(path) # posortowanie ścieżki

    return path, LB

In [7]:
def main():
    table1 = np.array([[inf, 7, 10, 9, 1, 7],
                      [10, inf, 3, 10, 11, 3],
                      [14, 2, inf, 10, 3, 3],
                      [3, 3, 8, inf, 12, 8],
                      [9, 8, 11, 8, inf, 2],
                      [11, 9, 5, 12, 9, inf]])

    table2 = np.array([[inf, 10, 8, 19, 12],
                      [10, inf, 20, 6, 3],
                      [8, 20, inf, 4, 2],
                      [19, 6, 4, inf, 7],
                      [12, 3, 2, 7, inf]])

    print_matrix(table1)
    path1, lower_bound1 = TSP(table1)
    print("\nŚcieżka:", path1)
    print("dolne ograniczenie:", lower_bound1, "\n\n\n")

    print_matrix(table2)
    path2, lower_bound2 = TSP(table2)
    print("\nŚcieżka:", path2)
    print("dolne ograniczenie:", lower_bound2)

main()

    |       0      1      2      3      4      5

------------------------------------------------
  0 |     inf    7.0   10.0    9.0    1.0    7.0

  1 |    10.0    inf    3.0   10.0   11.0    3.0

  2 |    14.0    2.0    inf   10.0    3.0    3.0

  3 |     3.0    3.0    8.0    inf   12.0    8.0

  4 |     9.0    8.0   11.0    8.0    inf    2.0

  5 |    11.0    9.0    5.0   12.0    9.0    inf


Macierz
 [[inf  6.  9.  2.  0.  6.]
 [ 7. inf  0.  1.  8.  0.]
 [12.  0. inf  2.  1.  1.]
 [ 0.  0.  5. inf  9.  5.]
 [ 7.  6.  9.  0. inf  0.]
 [ 6.  4.  0.  1.  4. inf]]
LB: 22.0

Macierz
 [[ 6.  9. inf  0.  6.]
 [inf  0.  1.  8.  0.]
 [ 0. inf  2.  1.  1.]
 [ 6.  9.  0. inf  0.]
 [ 4.  0.  1.  4. inf]]
LB: 22.0

Macierz
 [[inf  0.  1.  0.]
 [ 0. inf  2.  1.]
 [inf  9.  0.  0.]
 [ 4.  0.  1. inf]]
LB: 22.0

Macierz
 [[inf  1.  0.]
 [ 9.  0.  0.]
 [ 0.  1. inf]]
LB: 22.0

Macierz
 [[ 0. inf]
 [ 0.  0.]]
LB: 23.0

Ścieżka: [(3, 0), (0, 4), (4, 5), (5, 2), (2, 1), (1, 3)]
dolne ograniczenie: 23

Poniżej zamieszczamy ręczne rozwiązanie obu obliczonych przykładów, co potwierdza poprawność naszej implementacji algorytmu.

<img src="obliczenia.png">

**Wnioski**
  
Algorytm Little'a dla problemu komiwojażera okazał się dużo bardziej złożony niż implementowane wcześniej podejścia zachłanne. Mimo to, dzięki pracy grupowej udało się nam zrealizować w pełni działający algorytm. Ta wersja rozwiązania problemu wyróżnia się tym, że zwraca ona *wprost* trasę optymalną, a nie jedynie jej *aproksymację*. Ze względu na jego złożoność i jej eksponencjalny wzrost razem ze zwiększaniem liczby wierzchołków grafu (miast do odwiedzenia), nie jest opłacalne użycie tego algorytmu dla problemów o większej liczbe miast (np. 40).